# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!nvidia-smi -L

In [ ]:
!pip install accelerate

### Setup the model

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
def load_model():
    try:
        model = AutoModelForCausalLM.from_pretrained(
            "NousResearch/Llama-2-7b-chat-hf",
            load_in_8bit=True,
            device_map='auto',
        )
        return model
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
model = load_model()
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

### Freezing the original weights


In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

## Data

In [ ]:
import datasets
from transformers import Trainer

In [ ]:
data = datasets.load_dataset("csv", data_files="%data%")
data['train']['text'][:5]

In [ ]:
data['train'][0]

In [ ]:
data = data.map(lambda samples: tokenizer(samples['text']), batched=True)
data

### Training

In [ ]:
import transformers

# Create a data collator for language modeling.
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

def train_model():
    # Create a trainer.
    trainer = Trainer(
        model=model,
        train_dataset=data['train'],
        args=transformers.TrainingArguments(
            per_device_train_batch_size=2,
            gradient_accumulation_steps=5,
            warmup_steps=50,
            max_steps=100,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=1,
            output_dir='%name%'
        ),
        data_collator=data_collator,
    )
    
    # Silence the warnings.
    model.config.use_cache = True
    trainer.train()
    
    return trainer

trainer = train_model()

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'trainer' is your Trainer object
trainer_state = trainer.state

# Extract the loss values
loss_values = [entry['loss'] for entry in trainer_state.log_history[:-1]]

# Create a range of steps
steps = list(range(len(loss_values)))

# Plot the loss values
plt.figure(figsize=(10, 5))
plt.plot(steps, loss_values, label='loss')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
trainer.save_model('%name%')

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "%name%"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, add_eos_token=True)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
model.save_pretrained("%name%")

## Share adapters on the 🤗 Hub

In [ ]:
model.push_to_hub("%name%",
                  use_auth_token=True,
                  commit_message="new files")

## Load adapters from the Hub

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "%name%"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

## Inference

In [ ]:
batch = tokenizer("“potatoe are big ” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))